# Handling Imbalanced Dataset with Random Oversampling

In [ ]:
pip install -U imbalanced-learn

In [ ]:
import imblearn
from imblearn.over_sampling import RandomOverSampler 

In [ ]:
oversample = RandomOverSampler(sampling_strategy='minority',random_state=42)

In [ ]:
df2_imb=urdu_news.copy()

In [ ]:
df2_imb.head(2)

In [ ]:
df2_imb.dropna(axis=1,inplace=True)

In [ ]:
df2_imb['Date'] = pd.to_datetime(df2_imb['Date'])

In [ ]:
df2_imb['Class'].value_counts().reset_index()

In [ ]:
# Text Normalization
df2_imb['News Headlines'] = df2_imb['News Headlines'].apply(normalize)
# Text normalize characters
df2_imb['News Headlines'] = df2_imb['News Headlines'].apply(normalize_characters)
# Text normalize combined characters
df2_imb['News Headlines'] = df2_imb['News Headlines'].apply(normalize_combine_characters)
# remove_diacritics
df2_imb['News Headlines'] = df2_imb['News Headlines'].apply(remove_diacritics)
df2_imb['News Headlines'] = df2_imb['News Headlines'].apply(normalize_whitespace) 
df2_imb['News Headlines'] = df2_imb['News Headlines'].apply(preprocess) 
df2_imb['News Headlines']= df2_imb['News Headlines'].apply(remove_punctuation) 
df2_imb['News Headlines'] = df2_imb['News Headlines'].apply(remove_accents)
df2_imb['News Headlines'] = df2_imb['News Headlines'].apply(replace_urls) 
df2_imb['News Headlines']= df2_imb['News Headlines'].apply(replace_numbers) 
df2_imb['News Headlines'] = df2_imb['News Headlines'].apply(replace_emails) 
df2_imb['News Headlines'] = df2_imb['News Headlines'].apply(replace_currency_symbols) 
df2_imb['News Headlines'] = df2_imb['News Headlines'].apply(normalize_whitespace) 
df2_imb['News Headlines'] = df2_imb['News Headlines'].apply(digits_space)

In [ ]:
df2_imb['lemmatized_text'] = df2_imb['News Headlines'].apply(lemitizeStr)

In [ ]:
df2_imb = df2_imb.sample(frac=1,random_state=42)
df2_imb.reset_index(drop=True)

In [ ]:
split_index_1_imb = int(len(df2_imb) * 0.8)
split_index_2_imb = int(len(df2_imb) * 0.9)

In [ ]:
train_df2_imb, val_df2_imb, test_df2_imb = df2_imb [:split_index_1_imb],df2_imb [split_index_1_imb:split_index_2_imb],
df2_imb[split_index_2_imb:]

In [ ]:
test_df2=test_df2_imb.copy()

In [ ]:
train_df2_imb['Class'].value_counts().reset_index()

In [ ]:
x_train_df2_imb=train_df2_imb.drop(columns=['Class'])
y_train_df2_imb=train_df2_imb[['Class']]

In [ ]:
train_x, train_y = oversample.fit_resample(x_train_df2_imb, y_train_df2_imb)

In [ ]:
train_df2_sampled=pd.concat([train_x, train_y],axis=1)

In [ ]:
train_df2_sampled['Class'].value_counts()

In [ ]:
train_df2_sampled.rename(str.lower, axis='columns',inplace=True)
val_df2_imb.rename(str.lower, axis='columns',inplace=True)
test_df2.rename(str.lower, axis='columns',inplace=True)

In [ ]:
temp1 = train_df2_sampled.groupby('class').count()['labels'].reset_index().sort_values(by='class',ascending=True)
temp1.style.background_gradient(cmap='Purples')

In [ ]:
fig = go.Figure(go.Funnelarea(
    text = temp1['class'],
    values = temp1.labels,
    title = {"position": "top center", "text": " Train Sentiment Class Distribution"}
    ))
fig.show()

In [ ]:
temp2 = val_df2_imb.groupby('class').count()['labels'].reset_index().sort_values(by='class',ascending=True)
temp2.style.background_gradient(cmap='Purples')

In [ ]:
fig = go.Figure(go.Funnelarea(
    text = temp2['class'],
    values = temp2.labels,
    title = {"position": "top center", "text": " Validation Sentiment Class Distribution"}
    ))
fig.show()

In [ ]:
temp3 = test_df2.groupby('class').count()['labels'].reset_index().sort_values(by='class',ascending=True)
temp3.style.background_gradient(cmap='Purples')
fig = go.Figure(go.Funnelarea(
    text = temp3['class'],
    values = temp3.labels,
    title = {"position": "top center", "text": " Validation Sentiment Class Distribution"}
    ))
fig.show()

In [ ]:
Batch_Size=4

In [ ]:
encoder = LabelEncoder()
encoder.fit(df2_imb['Class'].tolist())

y_train_df2_sampled = encoder.transform(train_df2_sampled['class'].tolist())
y_test_df2_sampled = encoder.transform(test_df2['class'].tolist())
y_val_df2_sampled = encoder.transform(val_df2_imb['class'].tolist())


y_train_df2_sampled_1 = y_train_df2_sampled.reshape(-1,1)
y_test_df2_sampled_1 = y_test_df2_sampled.reshape(-1,1)
y_val_df2_sampled_1 = y_val_df2_sampled.reshape(-1,1)

print("y_train",y_train_df2_sampled_1.shape)
print("y_test",y_test_df2_sampled_1.shape)
print("y_val",y_val_df2_sampled_1.shape)

In [ ]:
train_encoded_sampled_1=train_df2_sampled.lemmatized_text.tolist()
val_encoded_sampled_1=val_df2_imb.lemmatized_text.tolist()
test_encoded_sampled_1=test_df2.lemmatized_text.tolist()
train_encoded_sampled_1 = robert_encode(train_encoded_sampled_1)
val_encoded_sampled_1 = robert_encode(val_encoded_sampled_1)
test_encoded_sampled_1 = robert_encode(test_encoded_sampled_1)

In [ ]:
train_encoded_sampled = bert_encode(train_df2_sampled.lemmatized_text)
val_encoded_sampled = bert_encode(val_df2_imb.lemmatized_text)
test_encoded_sampled = bert_encode(test_df2.lemmatized_text)

In [ ]:
train_dataset_sampled = (
    tf.data.Dataset
    .from_tensor_slices((train_encoded_sampled, y_train_df2_sampled))
    .shuffle(1000)
    .batch(Batch_Size,drop_remainder=True)
)

val_dataset_sampled = (
    tf.data.Dataset
    .from_tensor_slices((val_encoded_sampled, y_val_df2_sampled))
    .shuffle(1000)
    .batch(Batch_Size,drop_remainder=True)
)

test_dataset_sampled = (
    tf.data.Dataset
    .from_tensor_slices((test_encoded_sampled, y_test_df2_sampled))
    .batch(Batch_Size)
)

In [ ]:
train_dataset_sampled_1 = (
    tf.data.Dataset
    .from_tensor_slices((train_encoded_sampled_1, y_train_df2_sampled_1))
    .shuffle(1000)
    .batch(Batch_Size,drop_remainder=True)
)

val_dataset_sampled_1 = (
    tf.data.Dataset
    .from_tensor_slices((val_encoded_sampled_1, y_val_df2_sampled_1))
    .shuffle(1000)
    .batch(Batch_Size,drop_remainder=True)
)

test_dataset_sampled_1 = (
    tf.data.Dataset
    .from_tensor_slices((test_encoded_sampled_1, y_test_df2_sampled_1))
    .batch(Batch_Size)
)

In [ ]:
print('======Train dataset ====')
for element in train_dataset_sampled.as_numpy_iterator(): 
    print(element) 

## BERT

In [ ]:
model_sampled = bert_model(model_prembert)
adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.002)
acc=tf.keras.metrics.SparseCategoricalAccuracy(
    name="accuracy")

model_sampled.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False,name="sparse_categorical_crossentropy"),
              optimizer=adam_optimizer,metrics=[acc])

In [ ]:
%%time
 history_sampled = model_sampled.fit(
                 train_dataset_sampled,
                     batch_size=Batch_Size,epochs=50,validation_data=val_dataset_sampled,
                     callbacks = callbacks)

In [ ]:
print("Evaluate")
result_model_sampled = model_sampled.evaluate(test_dataset_sampled, batch_size=Batch_Size)
dict(zip(model_sampled.metrics_names, result_model_sampled))

In [ ]:
model_sampled_pred = model_sampled.predict(test_dataset_sampled,batch_size=Batch_Size)
model_sampled_pred

In [ ]:
if os.path.isfile('Bert_model_sampled/Bert_model_sampled.h5') is False:
     model_sampled.save('Bert_model_sampled/Bert_model_sampled.h5')

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(10,7))
ax[0].plot(history_sampled.history['loss'], color='b', label="Training loss")
ax[0].plot(history_sampled.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)
ax[1].plot(history_sampled.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(history_sampled.history['val_accuracy'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)
plt.title(label='Bert-Multilingual Sampled',loc='center')
plt.ylabel("accuracy")
plt.xlabel("Epochs")

In [ ]:
classes_bert_sampled = np.argmax(model_sampled_pred, axis=-1)
classes_bert_sampled=classes_bert_sampled.reshape(-1,1)
df_actual_sampled = pd.DataFrame(y_test_df2_sampled, columns = ['actual'])
df_predicted_bert_sampled = pd.DataFrame(classes_bert_sampled, columns = ['predicted'])
final_bert_sampled=pd.concat([df_actual_sampled , df_predicted_bert_sampled],axis=1)
test_pred_output_bert_sampled=pd.concat([test_df2.reset_index(drop=True), final_bert_sampled],axis=1)
test_pred_output_bert_sampled.head(5)

In [ ]:
target_names = ['Negative', 'Neutral', 'Positive']
print(classification_report(y_test_df2_sampled, classes_bert_sampled,target_names=target_names))

In [ ]:
cm_sampled = confusion_matrix(y_test_df2_sampled, classes_bert_sampled)
cmn_2 = cm_sampled.astype('float') / cm.sum(axis=1)[:, np.newaxis]
df_cm_sampled= pd.DataFrame(cmn_2, index=target_names, columns=target_names)
show_confusion_matrix(df_cm_sampled)

## Roberta Urdu

In [ ]:
model_roberta_sampled = bert_model(model_preroberta)
adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.002)
acc=tf.keras.metrics.SparseCategoricalAccuracy(
    name="accuracy")

model_roberta_sampled.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False,name="sparse_categorical_crossentropy"),
              optimizer=adam_optimizer,metrics=[acc])

In [ ]:
%%time
history_roberta_sampled = model_roberta_sampled.fit(
    train_dataset_sampled_1,
    batch_size=Batch_Size,
    epochs=50,
    validation_data=val_dataset_sampled_1,
    callbacks = callbacks)

In [ ]:
print("Evaluate")
result_roberta_sampled = model_roberta_sampled.evaluate(test_dataset_sampled_1, batch_size=Batch_Size)
dict(zip(model_roberta_sampled.metrics_names, result_roberta_sampled))

In [ ]:
model_roberta_sampled_pred = model_roberta_sampled.predict(test_dataset_sampled_1,batch_size=Batch_Size)
model_roberta_sampled_pred

In [ ]:
if os.path.isfile('Roberta_sampled/Roberta_sampled.h5') is False:
     model_roberta_sampled.save('Roberta_sampled/Roberta_sampled.h5')

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(10,7))
ax[0].plot(history_roberta_sampled.history['loss'], color='b', label="Training loss")
ax[0].plot(history_roberta_sampled.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)
ax[1].plot(history_roberta_sampled.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(history_roberta_sampled.history['val_accuracy'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)
plt.title(label='Roberta Urdu Sampled',loc='center')
plt.ylabel("accuracy")
plt.xlabel("Epochs")

In [ ]:
classes_roberta_sampled_1 = np.argmax(model_roberta_sampled_pred, axis=-1)
classes_roberta_sampled_1=classes_roberta_sampled_1.reshape(-1,1)
df_actual_sampled_1 = pd.DataFrame(y_test_df2_sampled_1, columns = ['actual'])
df_predicted_roberta_sampled = pd.DataFrame(classes_roberta_sampled_1, columns = ['predicted'])
final_roberta_sampled=pd.concat([df_actual_sampled_1, df_predicted_roberta_sampled],axis=1)
test_pred_output_robert_sampled=pd.concat([test_df2.reset_index(drop=True), final_roberta_sampled],axis=1)
test_pred_output_robert_sampled.head(5)

In [ ]:
from sklearn.metrics import classification_report
target_names = ['Negative', 'Neutral', 'Positive']
print(classification_report(y_test_df2_sampled_1, classes_roberta_sampled_1,target_names=target_names))

In [ ]:
cm_sampled_1 = confusion_matrix(y_test_df2_sampled_1, classes_roberta_sampled_1)
cmn_3 = cm_sampled_1.astype('float') / cm.sum(axis=1)[:, np.newaxis]
df_cm_sampled_1= pd.DataFrame(cmn_3, index=target_names, columns=target_names)
show_confusion_matrix(df_cm_sampled_1)

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
f1_score(y_test_df2_sampled_1, classes_roberta_sampled_1, average='micro')